# HuggingFacePipeline


In [1]:
from langchain.llms.huggingface_pipeline import HuggingFacePipeline
from langchain.prompts import PromptTemplate

prompt = PromptTemplate.from_template("A {word} is a")

llm = HuggingFacePipeline.from_model_id(
    model_id="gpt2",
    task="text-generation",
    pipeline_kwargs={"max_new_tokens": 50},
)

chain = prompt | llm

print(chain.invoke({"word": "tomato"}))

/Users/jjack/Documents/fullstack-gpt/env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/jjack/Documents/fullstack-gpt/env/lib/python3.11/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/Users/jjack/Documents/fullstack-gpt/env/lib/python3.11/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


 large tomato in nature.

This book contains all of the following facts:

* A small tomato is a medium to large tomato plant found in every vegetable family in the world;

* The plant is native to the eastern Mediterranean region


In [2]:
print(chain.invoke({"word": "potato"}))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


 small tree of sorts that may include five seed bulbs, a potato vine, an almond seedling, or any of many thousands which may contain several seeds. They do not grow in the same manner as apples or bananas which are small trees.




# HuggingFaceHub


In [26]:
from langchain.llms import HuggingFaceEndpoint
from langchain.llms import HuggingFaceHub
from langchain.prompts import PromptTemplate

prompt = PromptTemplate.from_template("What's the meaning of {word}")
prompt = PromptTemplate.from_template("[INST]What's the meaning of {word}[/INST]")

In [31]:
llm = HuggingFaceHub(
    # repo_id="mistralai/Mistral-7B-Instruct-v0.1",
    repo_id="mistralai/Mistral-7B-Instruct-v0.3",
    model_kwargs={
        "max_new_tokens": 250,
    },
)

llm.client.api_url = (
    # "https://api-inference.huggingface.co/models/mistralai/Mistral-7B-Instruct-v0.1"
    "https://api-inference.huggingface.co/models/mistralai/Mistral-7B-Instruct-v0.3"
)

chain = prompt | llm
print(chain.invoke({"word": "potato"}))

A potato is a starchy, tuberous crop from the Solanum tuberosum species. It is native to the Andean region of South America and is a major food source worldwide. Potatoes are usually cooked and eaten as a vegetable, and they can be prepared in various ways, such as boiling, baking, frying, or mashing. They are rich in carbohydrates, fiber, and vitamins, particularly vitamin C.
